# Spatial Feature Engineering (II)

## Map Synthesis

## 📖 Ahead of time...

In this second part of Spatial Feature Engineering, we turn to Map Synthesis. There is only one reading to complete for this block, [Chapter 12](https://geographicdata.science/book/notebooks/12_feature_engineering.html) of the GDS Book {cite}`reyABwolf`. This block of Spatial Feature Engineering in this course loosely follows the second part of the chapter ([Map Synthesis](https://geographicdata.science/book/notebooks/12_feature_engineering.html#feature-engineering-using-map-synthesis)).

## 💻 Hands-on coding

"Dataset enhancement through space":

- [ ] Distance buffer counts (within and between datasets)
- [ ] Ring buffers (within and between datasets)
- [ ] Clustering as map synthesis (example with DBSCAN and LISAs)

## 🐾 Next steps